In [6]:
import pandas as pd
import folium

pd.set_option('display.precision', 2)


data_path_feb = "../Data/01_Courses_usagers/2023_02/"
data_path_jun = "../Data/01_Courses_usagers/2023_06/"

render_map_path = "render_map/"

In [3]:
stations = pd.read_csv(data_path_feb + "All_data_courses.csv", sep=";").loc[:, ['from_stat', 'Latitude_start', 'longitude_start']].drop_duplicates()
stations = stations.rename(columns={"from_stat": "nom", "Latitude_start": "lat", "longitude_start": "long"})

print(stations)

                                               nom    lat  long
0                        Jean Moulin - Paul Doumer  48.86  2.46
1                          Lucie Aubrac - Franklin  48.86  2.44
2             Docteurs Déjérine - Pte de Montreuil  48.85  2.41
3                          Jules Vallès - Charonne  48.85  2.38
5        Gare de l'Est - Place du 11 Novembre 1918  48.88  2.36
...                                            ...    ...   ...
2249988                   Quai Panhard et Levassor  48.83  2.38
2282809                  Ney - Porte de Montmartre  48.90  2.34
2655446                      Jean Macé - Faidherbe  48.85  2.38
2790915                        Centenaire - Sorins  48.86  2.43
2972450              Stalingrad - Désiré Chevalier  48.86  2.44

[1471 rows x 3 columns]


In [4]:
avg_loc = stations[['lat', 'long']].mean().to_list()
map_paris = folium.Map(locaion=avg_loc, zoom_start=15)

for station in stations.itertuples():
    marker = folium.CircleMarker(location=(float(station.lat), float(station.long)), tooltip=station.nom, radius=2, weight=2)
    marker.add_to(map_paris)
    
sw = stations[['lat', 'long']].min().values.tolist()
ne = stations[['lat', 'long']].max().values.tolist()
map_paris.fit_bounds([sw, ne])
map_paris

## Trajets pour Février 2023

In [14]:
courses_02 = pd.read_csv(data_path_feb + "All_data_courses.csv", sep=";").loc[:, ['id_velo', 'date_depart', 'date_arrive', 'from_stat', 'to_stat']]
courses_06 = pd.read_csv(data_path_jun + "All_data_courses.csv").loc[:, ['id_velo', 'date_depart', 'date_arrive', 'from_stat', 'to_stat']]